In [17]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time
mixer.init()
beep = mixer.Sound('C:/Users/purib/Downloads/Drowsiness detection/Drowsiness detection/alarm.wav')
face = cv2.CascadeClassifier('C:/Users/purib/Downloads/Drowsiness detection/Drowsiness detection/haar cascade files/haarcascade_frontalface_alt.xml')
lefteye = cv2.CascadeClassifier('C:/Users/purib/Downloads/Drowsiness detection/Drowsiness detection/haar cascade files/haarcascade_lefteye_2splits.xml')
righteye = cv2.CascadeClassifier('C:/Users/purib/Downloads/Drowsiness detection/Drowsiness detection/haar cascade files/haarcascade_righteye_2splits.xml')
lbl=['Close','Open']
lbr=['closed','opened']
model = load_model('C:/Users/purib/Downloads/Drowsiness detection/Drowsiness detection/models/cnnCat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]
while(cap.isOpened()):
    ret, frame  = cap.read()
    height,width , channels = frame.shape
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = lefteye.detectMultiScale(gray)
    right_eye = righteye.detectMultiScale(gray)
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )
    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        right_pred = model.predict(r_eye) 
        
        if(right_pred.any()==1):
            lbr='opened'
        if(right_pred.any()==0):
            lbr='closed'
        break
    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        left_pred = model.predict(l_eye)
        
        if(left_pred.any()==1):
            lbl='O'
        if(left_pred.any()==0):
            lbl='shut'
        break
    if(lbr=='closed' and lbl=='shut'):
        score=score+1
        cv2.putText(frame,"sleep",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    # if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"awake",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score<0):
        score=0
    cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>40):
        #if the score is more than 40 it implies that the driver is asleep annd thus the alarm beeps
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            beep.play()
        except: # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step


KeyboardInterrupt: 